In [32]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from math import sqrt
import ROOT
import ctypes
try:
#     plt.style.use('belle2')
    plt.style.use('belle2_serif')
#     plt.style.use('belle2_modern')
except OSError:
    print("Please install belle2 matplotlib style") 
px = 1/plt.rcParams['figure.dpi']

from main.data_tools.extract_ntuples import get_pd, get_np
from main.draw_tools.decorations import b2helix, watermark
from main.draw_tools.stacking_with_error_bars import MC_stack_plot, MC_stack_plot_density

from main.data_tools.error_bars import make_data_weight
from main.data_tools.query_dataframes import cut_dfs_7types


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_dict = {r'$N_{topo}$': [684, 3634, 1283],
             'Fit': [681.225, 3697.640, 1305.752],
             'Pull': [-0.074, 0.891, 0.482] 
            }

In [3]:
df = pd.DataFrame(data_dict)

In [5]:
df.index = [r'$D^0 \to \phi \gamma$', r'$D^0 \to \phi \pi^0$', 'Others']
df

,$N_{topo}$,Fit,Pull
$D^0 \to \phi \gamma$,684,681.225,-0.074
$D^0 \to \phi \pi^0$,3634,3697.640,0.891
Others,1283,1305.752,0.482


In [ ]:
292.645

pull_1 = 

ref_data_dict = {r'$N_{topo}$': [684, 3634, 1283],
             'Fit': [70080.9, 3697.640, 1305.752],
             'Pull': [-0.074, 0.891, 0.482] 
            }

In [13]:
import csv
# CSV file name

def csv_to_dict(csv_file):
    # Initialize an empty dictionary
    data_dict = {}

    # Read the CSV file and convert it into a dictionary
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            for key, value in row.items():
                if key not in data_dict:
                    data_dict[key] = []
                data_dict[key].append(value)
    return data_dict




In [47]:
csv_file = '/media/jykim/T7/saved_plots/Measurement/phi/Fit_results.csv'
Fit_results_dict = csv_to_dict(csv_file)

N_Fit = Fit_results_dict['Fit']
Nsig = float(N_Fit[0])
print(Nsig)
Nsig_err = float(Fit_results_dict['Error'][0])

csv_file = '/media/jykim/T7/saved_plots/Measurement/phi/eff_D0_D0bar.csv'
eff_purity_multiplicity_dict = csv_to_dict(csv_file)

D0_eff = float(eff_purity_multiplicity_dict['Eff'][0])
D0_purity = float(eff_purity_multiplicity_dict['Purity'][0])
D0_signal_eff = D0_eff * D0_purity


D0bar_eff = float(eff_purity_multiplicity_dict['Eff'][1])
D0bar_purity = float(eff_purity_multiplicity_dict['Purity'][1])
D0bar_signal_eff = D0bar_eff * D0bar_purity


681.2254082330801


In [39]:
# csv_file = '/media/jykim/T7/saved_plots/Measurement/phi/ref_Fit_results.csv'
# Fit_results_dict = csv_to_dict(csv_file)

# N_Fit = Fit_results_dict['Fit']
# Nsig = float(N_Fit[0])
# print(Nsig)

csv_file = '/media/jykim/T7/saved_plots/Measurement/phi/ref_eff_D0_D0bar.csv'
eff_purity_multiplicity_dict = csv_to_dict(csv_file)

D0_ref_eff = float(eff_purity_multiplicity_dict['Eff'][0])
D0_ref_purity = float(eff_purity_multiplicity_dict['Purity'][0])
D0_signal_ref_eff = D0_ref_eff * D0_ref_purity


D0bar_ref_eff = float(eff_purity_multiplicity_dict['Eff'][1])
D0bar_ref_purity = float(eff_purity_multiplicity_dict['Purity'][1])
D0bar_signal_ref_eff = D0bar_ref_eff * D0bar_ref_purity

In [28]:
def cal_Br_Central_value(Nsig, Nref, eff_sig, eff_ref, Br_ref_PDG):
    print("Input values:")
    print("Nsig:", Nsig)
    print("Nref:", Nref)
    print("eff_sig:", eff_sig)
    print("eff_ref:", eff_ref)
    print("Br_ref_PDG:", Br_ref_PDG)
    
    N_real_sig = Nsig / eff_sig
    N_real_ref = Nref / eff_ref
    
    Br = Br_ref_PDG * N_real_sig / N_real_ref
    
    return Br

In [35]:
cal_Br_Central_value(681.225, 3.33616e+05,0.07387, 0.24497,4.08 /1000 )


Input values:
Nsig: 681.225
Nref: 333616.0
eff_sig: 0.07387
eff_ref: 0.24497
Br_ref_PDG: 0.00408


2.7627950734003363e-05

In [45]:

cal_Br_Central_value(Nsig, 334515, (D0_signal_eff+D0bar_signal_eff)/2, (D0_signal_ref_eff+D0bar_signal_ref_eff)/2, 4.08 /1000 )

# (2.7558395155650944e-05 - 2.80e-05)

Input values:
Nsig: 681.2254082330801
Nref: 334515
eff_sig: 0.07386499999999999
eff_ref: 0.244995
Br_ref_PDG: 0.00408


2.7558395155650944e-05

In [43]:
(2.7627950734003363e-05 - 2.80e-05)

-3.720492659966362e-07

In [53]:
from math import sqrt
def cal_Br_error_stat(  Nsig_err, Nsig, Nref_err, Nref, central_value):
    
    Variance =  (Nsig_err/Nsig)**2  \
                + (Nref_err/Nref)**2 
        
    
    TOTAL = sqrt(Variance)*central_value
    
    return TOTAL

In [65]:
central_value = cal_Br_Central_value(Nsig, 334515, (D0_signal_eff+D0bar_signal_eff)/2, (D0_signal_ref_eff+D0bar_signal_ref_eff)/2, 4.08 /1000 )


stat_unc = cal_Br_error_stat(  Nsig_err, Nsig, 675.376, 334515, central_value)

print(f'Central value = {central_value}')
print(f'statistical uncertainty = {stat_unc}')
print(f'pull = {(central_value - 2.80e-05)/stat_unc:.4f}')



Input values:
Nsig: 681.2254082330801
Nref: 334515
eff_sig: 0.07386499999999999
eff_ref: 0.244995
Br_ref_PDG: 0.00408
Central value = 2.7558395155650944e-05
statistical uncertainty = 1.5219074742752008e-06
pull = -0.2902


In [33]:
def cal_Br_error( Br_ref_PDG_err,Br_ref_PDG, Nsig_err, Nsig, Nref_err, Nref, eff_sig_err, eff_sig, eff_ref_err, eff_ref, central_value):
    
    Variance = (Br_ref_PDG_err/Br_ref_PDG)**2 + (Nsig_err/Nsig)**2 + (eff_sig_err/eff_sig)**2 \
                + (Nref_err/Nref)**2 + (eff_ref_err/eff_ref)**2
        
    
    TOTAL = sqrt(Variance)*central_value
    
    return TOTAL

In [34]:
Br_ref_PDG_err= 0.06 / 10**(3)
Br_ref_PDG= 4.06 / 10**(3)

Nsig_err= 26.983
Nsig= 681.225
Nref_err= 602.563
Nref= 333616
eff_sig_err=0.00026
eff_sig=0.07387
eff_ref_err=0.00030
eff_ref=0.24497


central_value = cal_Br_Central_value(681.225, 3.33616e+05,0.07387, 0.24497,4.08 /1000 )


cal_Br_error( Br_ref_PDG_err,Br_ref_PDG, Nsig_err, Nsig, Nref_err, Nref, eff_sig_err, eff_sig, eff_ref_err, eff_ref, central_value)




1.1736071632645529e-06

In [35]:
sqrt(2)

1.4142135623730951

## 